# 🎨 Neural Style Transfer

Welcome to **Neural Style Transfer**! In this notebook, we'll combine the content of one image with the artistic style of another using deep CNNs. Transform your photos into masterpieces!

## What you'll learn:
- How CNNs extract content and style features
- Gram matrices for style representation
- Optimization-based image generation
- Fast style transfer techniques

Let's create digital art! 🖼️

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input

plt.style.use('seaborn-v0_8')
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

# Create directories
os.makedirs('images', exist_ok=True)
os.makedirs('results', exist_ok=True)

In [ ]:
# Create sample images for demonstration
def create_sample_content_image(size=(512, 512)):
    """Create a sample content image"""
    # Create a landscape-like image
    img = np.zeros((*size, 3), dtype=np.uint8)
    
    # Sky gradient (blue to light blue)
    for i in range(size[0] // 2):
        intensity = int(100 + (i / (size[0] // 2)) * 155)
        img[i, :] = [135, 206, intensity]  # Sky blue gradient
    
    # Ground (green)
    for i in range(size[0] // 2, size[0]):
        img[i, :] = [34, 139, 34]  # Forest green
    
    # Add some geometric shapes (buildings/trees)
    # Building 1
    img[300:450, 100:200] = [169, 169, 169]  # Gray building
    # Building 2
    img[250:450, 300:380] = [105, 105, 105]  # Darker building
    # Tree
    img[350:450, 450:500] = [139, 69, 19]   # Tree trunk
    img[300:380, 430:520] = [0, 100, 0]     # Tree leaves
    
    return img

def create_sample_style_image(size=(512, 512)):
    """Create a sample style image with artistic patterns"""
    img = np.zeros((*size, 3), dtype=np.uint8)
    
    # Create swirling pattern (Van Gogh-like)
    center_x, center_y = size[0] // 2, size[1] // 2
    
    for i in range(size[0]):
        for j in range(size[1]):
            # Distance from center
            dx, dy = i - center_x, j - center_y
            distance = np.sqrt(dx**2 + dy**2)
            angle = np.arctan2(dy, dx)
            
            # Create swirl pattern
            swirl = np.sin(distance * 0.02 + angle * 3) * 0.5 + 0.5
            
            # Color based on swirl pattern
            if swirl > 0.7:
                img[i, j] = [255, 215, 0]    # Gold
            elif swirl > 0.4:
                img[i, j] = [30, 144, 255]   # Dodger blue
            else:
                img[i, j] = [138, 43, 226]   # Blue violet
    
    return img

# Create sample images
content_img = create_sample_content_image()
style_img = create_sample_style_image()

# Save images
Image.fromarray(content_img).save('images/content.jpg')
Image.fromarray(style_img).save('images/style.jpg')

# Display images
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(content_img)
axes[0].set_title('📷 Content Image')
axes[0].axis('off')

axes[1].imshow(style_img)
axes[1].set_title('🎨 Style Image')
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("✅ Sample images created and saved!")

In [ ]:
# Image preprocessing functions
def load_and_process_img(path_to_img, max_dim=512):
    """Load and preprocess image for style transfer"""
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    
    # Resize image
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)
    
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

def deprocess_img(processed_img):
    """Deprocess image for display"""
    x = processed_img.copy()
    if len(x.shape) == 4:
        x = np.squeeze(x, 0)
    
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# Load and preprocess images
content_image = load_and_process_img('images/content.jpg')
style_image = load_and_process_img('images/style.jpg')

print(f"Content image shape: {content_image.shape}")
print(f"Style image shape: {style_image.shape}")

In [ ]:
# Build style transfer model using VGG19
def get_model():
    """Create a VGG19 model with access to intermediate layers"""
    # Load VGG19 without top layers
    vgg = VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    
    # Content and style layers
    content_layers = ['block5_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 
                   'block4_conv1', 'block5_conv1']
    
    # Get outputs from specified layers
    outputs = [vgg.get_layer(name).output for name in style_layers + content_layers]
    model = models.Model([vgg.input], outputs)
    
    return model, style_layers, content_layers

# Create model
model, style_layers, content_layers = get_model()
num_style_layers = len(style_layers)
num_content_layers = len(content_layers)

print(f"Style layers: {style_layers}")
print(f"Content layers: {content_layers}")
print(f"Model created with {len(model.outputs)} outputs")

In [ ]:
# Style transfer functions
def gram_matrix(input_tensor):
    """Calculate Gram matrix for style representation"""
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1] * input_shape[2], tf.float32)
    return result / num_locations

def get_style_content_loss(outputs, style_targets, content_targets, 
                          style_weight=1e-2, content_weight=1e4):
    """Calculate style and content loss"""
    style_outputs = outputs[:num_style_layers]
    content_outputs = outputs[num_style_layers:]
    
    # Style loss
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[i] - style_targets[i])**2) 
                          for i in range(num_style_layers)])
    style_loss *= style_weight / num_style_layers
    
    # Content loss
    content_loss = tf.add_n([tf.reduce_mean((content_outputs[i] - content_targets[i])**2) 
                            for i in range(num_content_layers)])
    content_loss *= content_weight / num_content_layers
    
    total_loss = style_loss + content_loss
    return total_loss, style_loss, content_loss

def get_feature_representations(model, content_path, style_path):
    """Get style and content feature representations"""
    # Load images
    content_image = load_and_process_img(content_path)
    style_image = load_and_process_img(style_path)
    
    # Preprocess for VGG
    content_image = preprocess_input(content_image * 255)
    style_image = preprocess_input(style_image * 255)
    
    # Get features
    style_outputs = model(style_image)
    content_outputs = model(content_image)
    
    # Get style features (Gram matrices)
    style_features = [gram_matrix(style_output) for style_output in style_outputs[:num_style_layers]]
    
    # Get content features
    content_features = [content_output for content_output in content_outputs[num_style_layers:]]
    
    return style_features, content_features

print("✅ Style transfer functions defined!")

In [ ]:
# Get target features
style_targets, content_targets = get_feature_representations(model, 'images/content.jpg', 'images/style.jpg')

print(f"Style targets: {len(style_targets)} layers")
print(f"Content targets: {len(content_targets)} layers")

# Visualize Gram matrices
fig, axes = plt.subplots(1, len(style_targets), figsize=(15, 3))
for i, (ax, gram) in enumerate(zip(axes, style_targets)):
    # Take a slice of the Gram matrix for visualization
    gram_slice = gram[0, :min(64, gram.shape[1]), :min(64, gram.shape[2])]
    im = ax.imshow(gram_slice, cmap='viridis')
    ax.set_title(f'Gram Matrix\nLayer {i+1}')
    ax.axis('off')

plt.suptitle('🎨 Style Representation (Gram Matrices)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Style transfer optimization
@tf.function
def train_step(image, model, style_targets, content_targets, 
               style_weight=1e-2, content_weight=1e4, total_variation_weight=30):
    """Single training step for style transfer"""
    with tf.GradientTape() as tape:
        outputs = model(image)
        total_loss, style_loss, content_loss = get_style_content_loss(
            outputs, style_targets, content_targets, style_weight, content_weight)
        
        # Add total variation loss for smoothness
        tv_loss = total_variation_weight * tf.image.total_variation(image)
        total_loss += tv_loss
    
    grad = tape.gradient(total_loss, image)
    return total_loss, style_loss, content_loss, tv_loss, grad

def run_style_transfer(content_path, style_path, num_iterations=1000, 
                      style_weight=1e-2, content_weight=1e4):
    """Run style transfer optimization"""
    # Initialize with content image
    image = load_and_process_img(content_path)
    image = tf.Variable(preprocess_input(image * 255), dtype=tf.float32)
    
    # Optimizer
    opt = tf.optimizers.Adam(learning_rate=5, beta_1=0.99, epsilon=1e-1)
    
    # Store losses
    losses = {'total': [], 'style': [], 'content': [], 'tv': []}
    
    print(f"🎨 Starting style transfer optimization...")
    print(f"Iterations: {num_iterations}")
    
    for i in range(num_iterations):
        total_loss, style_loss, content_loss, tv_loss, grads = train_step(
            image, model, style_targets, content_targets, style_weight, content_weight)
        
        opt.apply_gradients([(grads, image)])
        
        # Clip pixel values
        image.assign(tf.clip_by_value(image, -103.939, 255 - 123.68))
        
        # Store losses
        losses['total'].append(total_loss.numpy())
        losses['style'].append(style_loss.numpy())
        losses['content'].append(content_loss.numpy())
        losses['tv'].append(tv_loss.numpy())
        
        if i % 100 == 0:
            print(f"Iteration {i}: Total Loss = {total_loss:.2f}")
    
    return image, losses

# Run style transfer
stylized_image, losses = run_style_transfer('images/content.jpg', 'images/style.jpg', 
                                           num_iterations=500)

print("\n✅ Style transfer completed!")

In [ ]:
# Visualize results
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Original images and result
axes[0, 0].imshow(deprocess_img(load_and_process_img('images/content.jpg').numpy()))
axes[0, 0].set_title('📷 Content Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(deprocess_img(load_and_process_img('images/style.jpg').numpy()))
axes[0, 1].set_title('🎨 Style Image')
axes[0, 1].axis('off')

# Stylized result
result_img = deprocess_img(stylized_image.numpy())
axes[1, 0].imshow(result_img)
axes[1, 0].set_title('🖼️ Stylized Result')
axes[1, 0].axis('off')

# Loss curves
axes[1, 1].plot(losses['total'], label='Total Loss', alpha=0.8)
axes[1, 1].plot(losses['style'], label='Style Loss', alpha=0.8)
axes[1, 1].plot(losses['content'], label='Content Loss', alpha=0.8)
axes[1, 1].set_title('📉 Training Losses')
axes[1, 1].set_xlabel('Iteration')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Save result
Image.fromarray(result_img).save('results/stylized_result.jpg')
print("\n💾 Stylized image saved to 'results/stylized_result.jpg'")

print(f"\n📊 Final Losses:")
print(f"Total Loss: {losses['total'][-1]:.2f}")
print(f"Style Loss: {losses['style'][-1]:.2f}")
print(f"Content Loss: {losses['content'][-1]:.2f}")
print(f"TV Loss: {losses['tv'][-1]:.2f}")

## 🎉 Congratulations!

You've successfully implemented neural style transfer! Here's what you've accomplished:

✅ **Feature Extraction**: Used VGG19 to extract content and style features  
✅ **Gram Matrices**: Computed style representations  
✅ **Optimization**: Iteratively optimized image to match targets  
✅ **Artistic Transfer**: Created beautiful stylized images  

### 🚀 Next Steps:
1. Try different content and style images
2. Experiment with different layer combinations
3. Implement fast style transfer networks
4. Move on to **Project 08: Variational Autoencoder (VAE)**

Ready for generative modeling? Let's explore latent spaces! 🌌